In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup
import sys
import os

import sklearn
from sklearn import cross_validation
from sklearn import svm
from sklearn import metrics

reload(sys)
#sys.setdefaultencoding("ISO-8859-1")
sys.setdefaultencoding("UTF-8")

In [2]:
def clean_text(raw_text):
    review_text = BeautifulSoup(raw_text).get_text()
    words = review_text.lower().split()
    return(" ".join(words))

In [3]:
languages = ["english"]
datafiles = ["summary-english-truth.txt"]
tasks = ["age"]

In [4]:
for language, datafile in zip(languages, datafiles):
    train = pd.read_csv(datafile, header=0, delimiter="\t", quoting=1)
    num_text = train["text"].size
    clean_train_data = []
    
    for i in xrange( 0, num_text):
        clean_train_data.append( clean_text( train["text"][i] ) )
    
    vectorizer = TfidfVectorizer(analyzer = "word", tokenizer = None, preprocessor = None)
    train_x = vectorizer.fit_transform(clean_train_data)
    train_x = train_x.toarray()
    print "shape: ", train_x.shape
    rows, cols = train_x.shape

In [5]:
#looking into information gain

In [6]:
with open('age-important-words-using-info-gain.txt') as f:
    alist = [line.rstrip() for line in f]

In [7]:
all_indices_ranked = alist[0].split(',')
all_indices_ranked = [int(x) for x in all_indices_ranked]
all_indices_ranked = [x-1 for x in all_indices_ranked]

In [8]:
top_info_words_numbers = [100, 200, 300, 500, 700, 1000, 2000, 5000, 7000, 8000, 9000, 10000, cols-1]
top_info_words_numbers =  sorted(top_info_words_numbers, reverse=True)
top_info_words_numbers

[26263, 10000, 9000, 8000, 7000, 5000, 2000, 1000, 700, 500, 300, 200, 100]

In [9]:
train = pd.read_csv("summary-english-truth.txt", header=0, delimiter="\t", quoting=1)
train_y = train['age']
list_of_scores = []
for num_info_words in top_info_words_numbers:
    clf = svm.SVC(kernel='linear', C=1)
    scoring_function = 'accuracy'
    xx = [all_indices_ranked[x] for x in range(0, num_info_words)]
    xx = tuple(xx)
    smaller_train_x = train_x[:, xx]

    scores = cross_validation.cross_val_score(clf, smaller_train_x, train_y, cv=10, scoring=scoring_function)
    list_of_scores.append(scores)

In [10]:
y = [x.mean() for x in list_of_scores]
y

[0.69124999999999992,
 0.63208333333333333,
 0.63208333333333333,
 0.61874999999999991,
 0.61874999999999991,
 0.61874999999999991,
 0.61874999999999991,
 0.61874999999999991,
 0.61874999999999991,
 0.61874999999999991,
 0.61874999999999991,
 0.61874999999999991,
 0.61874999999999991]

In [11]:
from scipy import stats
import pandas as pd

In [12]:
p_value_matrix = np.zeros((len(list_of_scores)))

In [13]:
i = range(0,len(list_of_scores))
for x, i  in zip(list_of_scores,i):
    z_stat, p_val = stats.ranksums(list_of_scores[0], x)
    p_value_matrix[i] = p_val

In [14]:
p_value_matrix

array([ 1.        ,  0.07566157,  0.07566157,  0.02113393,  0.02113393,
        0.02113393,  0.02113393,  0.02113393,  0.02113393,  0.02113393,
        0.02113393,  0.02113393,  0.02113393])

In [15]:
with open('age-important-words-using-gain-ratio.txt') as f:
    alist = [line.rstrip() for line in f]

In [16]:
all_indices_ranked = alist[0].split(',')
all_indices_ranked = [int(x) for x in all_indices_ranked]
all_indices_ranked = [x-1 for x in all_indices_ranked]

top_info_words_numbers = [100, 200, 300, 500, 700, 1000, 2000, 5000, 7000, 8000, 9000, 10000, cols-1]
top_info_words_numbers =  sorted(top_info_words_numbers, reverse=True)

In [17]:
train = pd.read_csv("summary-english-truth.txt", header=0, delimiter="\t", quoting=1)
train_y = train['age']
list_of_scores = []
for num_info_words in top_info_words_numbers:
    clf = svm.SVC(kernel='linear', C=1)
    scoring_function = 'accuracy'
    xx = [all_indices_ranked[x] for x in range(0, num_info_words)]
    xx = tuple(xx)
    smaller_train_x = train_x[:, xx]

    scores = cross_validation.cross_val_score(clf, smaller_train_x, train_y, cv=10, scoring=scoring_function)
    list_of_scores.append(scores)

In [18]:
y = [x.mean() for x in list_of_scores]
y

[0.69124999999999992,
 0.63208333333333333,
 0.63208333333333333,
 0.61874999999999991,
 0.61874999999999991,
 0.61874999999999991,
 0.61874999999999991,
 0.61874999999999991,
 0.61874999999999991,
 0.61874999999999991,
 0.61874999999999991,
 0.61249999999999993,
 0.39500000000000002]

In [19]:
p_value_matrix = np.zeros((len(list_of_scores)))

In [20]:
p_value_matrix

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [21]:
i = range(0,len(list_of_scores))
for x, i  in zip(list_of_scores,i):
    z_stat, p_val = stats.ranksums(list_of_scores[0], x)
    p_value_matrix[i] = p_val

In [22]:
p_value_matrix

array([  1.00000000e+00,   7.56615721e-02,   7.56615721e-02,
         2.11339281e-02,   2.11339281e-02,   2.11339281e-02,
         2.11339281e-02,   2.11339281e-02,   2.11339281e-02,
         2.11339281e-02,   2.11339281e-02,   2.11339281e-02,
         1.57052284e-04])

In [23]:
#checking polynomial kernel with 9000 features

In [24]:
languages = ["english"]
datafiles = ["summary-english-truth.txt"]
tasks = ["age"]
for language, datafile in zip(languages, datafiles):
    train = pd.read_csv(datafile, header=0, delimiter="\t", quoting=1)
    num_text = train["text"].size
    clean_train_data = []
    
    for i in xrange( 0, num_text):
        clean_train_data.append( clean_text( train["text"][i] ) )
    
    vectorizer = TfidfVectorizer(analyzer = "word", tokenizer = None, preprocessor = None)
    train_x = vectorizer.fit_transform(clean_train_data)
    train_x = train_x.toarray()
    print "shape: ", train_x.shape
    rows, cols = train_x.shape
